In [12]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook


In [2]:
import json
with open('/Users/hassaanasif/desktop/yelp_api.json') as f: 
    login = json.load(f)
login.keys()


dict_keys(['client-id', 'api-key'])

In [3]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [32]:
results = yelp_api.search_query(location='New York, NY',
                                       term='Bagels')
print(type(results))
results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [11]:
!pip install tqdm

In [33]:
LOCATION = 'NY,NY'
TERM = 'Bagels'

In [34]:
JSON_FILE = "Data/results_in_progress_NY_Bagels.json"
JSON_FILE


'Data/results_in_progress_NY_Bagels.json'

In [35]:
file_exists = os.path.isfile(JSON_FILE)

if file_exists == False:
    
    
    
    folder = os.path.dirname(JSON_FILE)
   
    if len(folder)>0:
        # creating folder
        os.makedirs(folder,exist_ok=True)
        
        
    
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  

else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_NY_Bagels.json not found. Saving empty list to file.


In [36]:
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [37]:
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()


dict_keys(['businesses', 'total', 'region'])

In [38]:
total_results = results['total']
total_results

8000

In [39]:
results_per_page = len(results['businesses'])
results_per_page


20

In [40]:
import time, math
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages


400

In [41]:
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [42]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [43]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/400 [00:00<?, ?it/s]

In [46]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [47]:
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_NY_Bagels.json already exists. Deleting previous file...
[i] Data/results_in_progress_NY_Bagels.json not found. Saving empty list to new file.
- 0 previous results found.


400

In [48]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/400 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [49]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,VrCCr45dhN-RRM107iptdg,russ-and-daughters-new-york,Russ & Daughters,https://s3-media4.fl.yelpcdn.com/bphoto/luU5lf...,False,https://www.yelp.com/biz/russ-and-daughters-ne...,2619,"[{'alias': 'bagels', 'title': 'Bagels'}, {'ali...",4.5,"{'latitude': 40.722589, 'longitude': -73.98828...",[delivery],$$,"{'address1': '179 E Houston St', 'address2': '...",+12124754880,(212) 475-4880,1972.496648
1,j1S3NUrkB3BVT49n_e76NQ,best-bagel-and-coffee-new-york,Best Bagel & Coffee,https://s3-media2.fl.yelpcdn.com/bphoto/9-8DVz...,False,https://www.yelp.com/biz/best-bagel-and-coffee...,4203,"[{'alias': 'bagels', 'title': 'Bagels'}, {'ali...",4.5,"{'latitude': 40.7522683, 'longitude': -73.9910...","[delivery, pickup]",$,"{'address1': '225 W 35th St', 'address2': '', ...",+12125644409,(212) 564-4409,5213.531501
2,foO2N-TrdPBO-dFn6M35TA,brooklyn-bagel-and-coffee-company-new-york-8,Brooklyn Bagel & Coffee Company,https://s3-media1.fl.yelpcdn.com/bphoto/2ssnQh...,False,https://www.yelp.com/biz/brooklyn-bagel-and-co...,221,"[{'alias': 'bagels', 'title': 'Bagels'}, {'ali...",4.5,"{'latitude': 40.73104, 'longitude': -73.9932199}","[delivery, pickup]",$,"{'address1': '63 E 8th St', 'address2': None, ...",+12124773070,(212) 477-3070,2846.031390
3,pb_ryc8FYHpUPV56eV2Eyg,brooklyn-bagel-and-coffee-company-new-york-2,Brooklyn Bagel & Coffee Company,https://s3-media2.fl.yelpcdn.com/bphoto/hUTrAB...,False,https://www.yelp.com/biz/brooklyn-bagel-and-co...,1762,"[{'alias': 'bagels', 'title': 'Bagels'}, {'ali...",4.5,"{'latitude': 40.746183, 'longitude': -73.997642}","[delivery, pickup]",$,"{'address1': '286 8th Ave', 'address2': '', 'a...",+12129242824,(212) 924-2824,4538.650359
4,oi39VAwo4-KGm_gSkWPCsQ,tompkins-square-bagels-avenue-a-new-york,Tompkins Square Bagels - Avenue A,https://s3-media2.fl.yelpcdn.com/bphoto/6yuU0v...,False,https://www.yelp.com/biz/tompkins-square-bagel...,1373,"[{'alias': 'bagels', 'title': 'Bagels'}]",4.0,"{'latitude': 40.72823, 'longitude': -73.9823}","[delivery, pickup]",$$,"{'address1': '165 Avenue A', 'address2': '', '...",+16463516520,(646) 351-6520,2727.847810


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,kb4ezfYAZiu_uEtANLGU2w,york-grill-and-juice-bar-new-york,York Grill & Juice Bar,https://s3-media3.fl.yelpcdn.com/bphoto/lWz61O...,False,https://www.yelp.com/biz/york-grill-and-juice-...,24,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",4.5,"{'latitude': 40.76608383097776, 'longitude': -...","[pickup, delivery]",NaN,"{'address1': '419 E 70th St', 'address2': '', ...",+19174093206,(917) 409-3206,7486.502240
996,0ETMN5wIvcS5TqeHJLBofA,bagels-and-beyond-brooklyn,Bagels and Beyond,https://s3-media3.fl.yelpcdn.com/bphoto/Nimx9P...,False,https://www.yelp.com/biz/bagels-and-beyond-bro...,29,"[{'alias': 'bagels', 'title': 'Bagels'}, {'ali...",4.0,"{'latitude': 40.590446, 'longitude': -73.973419}","[pickup, delivery]",$,"{'address1': '257 Ave X', 'address2': '', 'add...",+17183362396,(718) 336-2396,12904.910359
997,NK6HF-9zB2AQdApbcrqvyQ,kates-corner-cafe-astoria,Kate's Corner Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/wnbxCE...,False,https://www.yelp.com/biz/kates-corner-cafe-ast...,21,"[{'alias': 'cafes', 'title': 'Cafes'}]",4.5,"{'latitude': 40.77569, 'longitude': -73.90012}","[pickup, delivery]",$,"{'address1': '4125 20th Ave', 'address2': None...",+17189562929,(718) 956-2929,11134.066187
998,NMDxZsnglGMG7Q23TrtoQQ,johns-deli-brooklyn-6,John's Deli,https://s3-media4.fl.yelpcdn.com/bphoto/tAvg4A...,False,https://www.yelp.com/biz/johns-deli-brooklyn-6...,38,"[{'alias': 'convenience', 'title': 'Convenienc...",4.5,"{'latitude': 40.652248, 'longitude': -73.97563...","[pickup, delivery]",$$,"{'address1': '1269 Prospect Ave', 'address2': ...",+17188516200,(718) 851-6200,6120.777122
999,dKgMlvm_kcm6eLxCjkCwGw,forest-superette-and-bagels-staten-island,Forest superette and bagels,https://s3-media2.fl.yelpcdn.com/bphoto/W0ge81...,False,https://www.yelp.com/biz/forest-superette-and-...,5,"[{'alias': 'bagels', 'title': 'Bagels'}]",2.0,"{'latitude': 40.6308999, 'longitude': -74.10101}","[pickup, delivery]",$,"{'address1': '350 Forest Ave', 'address2': Non...",+17182736928,(718) 273-6928,12234.759914


In [50]:
final_df.duplicated(subset='id').sum()

0

In [31]:
final_df.to_csv('Data/Bagels_NY.csv.gz', compression='gzip',index=False)